In [1]:
from roboflow import Roboflow

rf = Roboflow(api_key="mZ6IIA1WPaLKIBOVETP5")
project = rf.workspace("joseph-nelson").project("thermal-dogs-and-people")
version = project.version(6)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Thermal-Dogs-and-People-6 in coco:: 100%|██████████| 211/211 [00:00<00:00, 2938.47it/s]


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets

import torchvision.transforms as transforms
from torchvision.models import vgg16
from torchvision.datasets import CocoDetection
import torchvision

import torchvision.models as models
import numpy as np

In [4]:
# load data

def collate_fn(batch):
    images, targets = zip(*batch)
    images = torch.stack([img for img in images], dim=0)
    return images, targets

# trans data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

coco_root = './Thermal-Dogs-and-People-6'  
train_img_folder = f'{coco_root}/train'  
train_ann_file = f'{coco_root}/train/_annotations.coco.json'  

# load COCO data
train_dataset = datasets.CocoDetection(root=train_img_folder, annFile=train_ann_file, transform=transform)

train_loader = DataLoader(
    train_dataset, 
    batch_size=8, 
    shuffle=True, 
    num_workers=1, 
    collate_fn=collate_fn
)
torch.cuda.empty_cache()

print(f'Dataloader length: {len(train_loader)}')  

# use GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()


def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, targets in dataloader:
            images = images.to(device)
                
            if all(len(target) > 0 for target in targets):  
                labels = torch.tensor([target[0]['category_id'] for target in targets], dtype=torch.long).to(device)
            else:
                continue

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            #print(f'Batch size: {labels.size(0)}, Total: {total}')  

        epoch_loss = running_loss / len(dataloader.dataset)

        if total > 0:
            accuracy = correct / total
        else:
            accuracy = 0  
            #print("Warning: No valid data processed in this epoch.")

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}')
    torch.cuda.empty_cache() 

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Dataloader length: 18


/home/seamonxiao/anaconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


# 1. CNN Models

## 1. ResNet-34

In [5]:
from torchvision.models import ResNet34_Weights

model = torchvision.models.resnet34(weights=ResNet34_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  

model = model.to(device)


optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 0.4883, Accuracy: 0.6613
Epoch [2/10], Loss: 0.5249, Accuracy: 0.7375
Epoch [3/10], Loss: 0.2152, Accuracy: 0.8953
Epoch [4/10], Loss: 0.1895, Accuracy: 0.7857
Epoch [5/10], Loss: 0.2084, Accuracy: 0.8333
Epoch [6/10], Loss: 0.1357, Accuracy: 0.8857
Epoch [7/10], Loss: 0.2399, Accuracy: 0.8000
Epoch [8/10], Loss: 0.2590, Accuracy: 0.8143
Epoch [9/10], Loss: 0.2170, Accuracy: 0.7917
Epoch [10/10], Loss: 0.1338, Accuracy: 0.8906


## 2. VGG-16

In [7]:
model = vgg16(weights='DEFAULT')
num_classes = 3
model.classifier[6] = nn.Linear(4096, num_classes)  

model = model.to(device)

'''
train_loader = DataLoader(
    train_dataset, 
    batch_size=8, 
    shuffle=True, 
    num_workers=0, 
    collate_fn=collate_fn
)
'''

optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, train_loader, criterion, optimizer, num_epochs=5)

Epoch [1/5], Loss: 5.3564, Accuracy: 0.5513
Epoch [2/5], Loss: 0.5753, Accuracy: 0.5286
Epoch [3/5], Loss: 0.4223, Accuracy: 0.5143
Epoch [4/5], Loss: 0.4793, Accuracy: 0.5106
Epoch [5/5], Loss: 0.4093, Accuracy: 0.4750


## 3. ResNet-50

In [5]:
model = models.resnet50(weights='DEFAULT')
model.fc = torch.nn.Linear(model.fc.in_features, len(train_dataset.coco.getCatIds())) 

model = model.to(device)


optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 0.3220, Accuracy: 0.6667
Epoch [2/10], Loss: 0.2614, Accuracy: 0.8488
Epoch [3/10], Loss: 0.1915, Accuracy: 0.8333
Epoch [4/10], Loss: 0.2065, Accuracy: 0.8488
Epoch [5/10], Loss: 0.3077, Accuracy: 0.7571
Epoch [6/10], Loss: 0.1751, Accuracy: 0.8889
Epoch [7/10], Loss: 0.2094, Accuracy: 0.8148
Epoch [8/10], Loss: 0.2612, Accuracy: 0.8191
Epoch [9/10], Loss: 0.1018, Accuracy: 0.9000
Epoch [10/10], Loss: 0.1002, Accuracy: 0.9359


# 2. Hyper-parameters

In [39]:
model = models.resnet50(weights='DEFAULT')
model.fc = torch.nn.Linear(model.fc.in_features, len(train_dataset.coco.getCatIds())) 

model = model.to(device)


optimizer = optim.Adam(model.parameters(), lr=0.001)

## 1. Learning Rate

### 1. 0.0001

In [11]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)

train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 0.6071, Accuracy: 0.5930
Epoch [2/10], Loss: 0.3551, Accuracy: 0.8333
Epoch [3/10], Loss: 0.2307, Accuracy: 0.9744
Epoch [4/10], Loss: 0.1068, Accuracy: 1.0000
Epoch [5/10], Loss: 0.0952, Accuracy: 0.9444
Epoch [6/10], Loss: 0.0608, Accuracy: 0.9861
Epoch [7/10], Loss: 0.0395, Accuracy: 1.0000
Epoch [8/10], Loss: 0.0512, Accuracy: 0.9872
Epoch [9/10], Loss: 0.0343, Accuracy: 0.9861
Epoch [10/10], Loss: 0.0156, Accuracy: 1.0000


### 2. 0.01

In [13]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 0.6931, Accuracy: 0.6000
Epoch [2/10], Loss: 0.5112, Accuracy: 0.4444
Epoch [3/10], Loss: 0.4587, Accuracy: 0.5897
Epoch [4/10], Loss: 0.3935, Accuracy: 0.5286
Epoch [5/10], Loss: 0.4012, Accuracy: 0.5694
Epoch [6/10], Loss: 0.4838, Accuracy: 0.4444
Epoch [7/10], Loss: 0.4036, Accuracy: 0.5769
Epoch [8/10], Loss: 0.2487, Accuracy: 0.6875
Epoch [9/10], Loss: 0.3615, Accuracy: 0.6364
Epoch [10/10], Loss: 0.2504, Accuracy: 0.7500


## 2. Optimizer Type

### 1. SGD

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)

train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 0.4832, Accuracy: 0.3906
Epoch [2/10], Loss: 0.4182, Accuracy: 0.5938
Epoch [3/10], Loss: 0.3795, Accuracy: 0.7571
Epoch [4/10], Loss: 0.3105, Accuracy: 0.8429
Epoch [5/10], Loss: 0.2841, Accuracy: 0.8000
Epoch [6/10], Loss: 0.2975, Accuracy: 0.7875
Epoch [7/10], Loss: 0.2214, Accuracy: 0.8000
Epoch [8/10], Loss: 0.2631, Accuracy: 0.7625
Epoch [9/10], Loss: 0.2099, Accuracy: 0.8750
Epoch [10/10], Loss: 0.1532, Accuracy: 0.9167


### 2. RMSprop

In [14]:
optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9, eps=1e-8)

train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 0.4899, Accuracy: 0.7051
Epoch [2/10], Loss: 0.3179, Accuracy: 0.7625
Epoch [3/10], Loss: 0.3646, Accuracy: 0.7361
Epoch [4/10], Loss: 0.3084, Accuracy: 0.8023
Epoch [5/10], Loss: 0.1533, Accuracy: 0.8857
Epoch [6/10], Loss: 0.2373, Accuracy: 0.7812
Epoch [7/10], Loss: 0.2125, Accuracy: 0.8281
Epoch [8/10], Loss: 0.1490, Accuracy: 0.9167
Epoch [9/10], Loss: 0.2296, Accuracy: 0.8333
Epoch [10/10], Loss: 0.2754, Accuracy: 0.8023


## 3. Regularization

### 1. 0.01

In [21]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)

train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 0.3159, Accuracy: 0.6452
Epoch [2/10], Loss: 0.2558, Accuracy: 0.8511
Epoch [3/10], Loss: 0.2120, Accuracy: 0.9000
Epoch [4/10], Loss: 0.2885, Accuracy: 0.7875
Epoch [5/10], Loss: 0.3699, Accuracy: 0.7872
Epoch [6/10], Loss: 0.2732, Accuracy: 0.7500
Epoch [7/10], Loss: 0.2381, Accuracy: 0.8143
Epoch [8/10], Loss: 0.3874, Accuracy: 0.6538
Epoch [9/10], Loss: 0.2850, Accuracy: 0.7639
Epoch [10/10], Loss: 0.2693, Accuracy: 0.7436


### 2. 0.001

In [25]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 0.3933, Accuracy: 0.6410
Epoch [2/10], Loss: 0.1983, Accuracy: 0.7429
Epoch [3/10], Loss: 0.1344, Accuracy: 0.8837
Epoch [4/10], Loss: 0.1589, Accuracy: 0.9231
Epoch [5/10], Loss: 0.2683, Accuracy: 0.8605
Epoch [6/10], Loss: 0.1486, Accuracy: 0.8974
Epoch [7/10], Loss: 0.2136, Accuracy: 0.9000
Epoch [8/10], Loss: 0.2037, Accuracy: 0.8721
Epoch [9/10], Loss: 0.1873, Accuracy: 0.8143
Epoch [10/10], Loss: 0.2188, Accuracy: 0.8625


## 4. Batch Size

### 1. 4

In [32]:
train_loader = DataLoader(
    train_dataset, 
    batch_size=4, 
    shuffle=True, 
    num_workers=1, 
    collate_fn=collate_fn
)

train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 0.5487, Accuracy: 0.6100
Epoch [2/10], Loss: 0.3953, Accuracy: 0.7959
Epoch [3/10], Loss: 0.3530, Accuracy: 0.7736
Epoch [4/10], Loss: 0.3232, Accuracy: 0.8529
Epoch [5/10], Loss: 0.4816, Accuracy: 0.7353
Epoch [6/10], Loss: 0.4022, Accuracy: 0.7451
Epoch [7/10], Loss: 0.2815, Accuracy: 0.8000
Epoch [8/10], Loss: 0.3579, Accuracy: 0.7255
Epoch [9/10], Loss: 0.3036, Accuracy: 0.8235
Epoch [10/10], Loss: 0.2576, Accuracy: 0.8396


### 2. 16

In [36]:
train_loader = DataLoader(
    train_dataset, 
    batch_size=16, 
    shuffle=True, 
    num_workers=1, 
    collate_fn=collate_fn
)

train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 0.2198, Accuracy: 0.5938
Epoch [2/10], Loss: 0.1440, Accuracy: 0.6250
Epoch [3/10], Loss: 0.0962, Accuracy: 0.9167
Epoch [4/10], Loss: 0.2201, Accuracy: 0.8710
Epoch [5/10], Loss: 0.0393, Accuracy: 0.9688
Epoch [6/10], Loss: 0.0213, Accuracy: 0.9688
Epoch [7/10], Loss: 0.0059, Accuracy: 1.0000
Epoch [8/10], Loss: 0.1178, Accuracy: 0.8750
Epoch [9/10], Loss: 0.0665, Accuracy: 0.9375
Epoch [10/10], Loss: 0.0216, Accuracy: 0.9688


### 3. 32

In [41]:
train_loader = DataLoader(
    train_dataset, 
    batch_size=32, 
    shuffle=True, 
    num_workers=1, 
    collate_fn=collate_fn
)

train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch [1/10], Loss: 0.0000, Accuracy: 0.0000
Epoch [2/10], Loss: 0.0000, Accuracy: 0.0000
Epoch [3/10], Loss: 0.2416, Accuracy: 0.3125
Epoch [4/10], Loss: 0.0665, Accuracy: 0.8571
Epoch [5/10], Loss: 0.0000, Accuracy: 0.0000
Epoch [6/10], Loss: 0.0000, Accuracy: 0.0000
Epoch [7/10], Loss: 0.0000, Accuracy: 0.0000
Epoch [8/10], Loss: 0.0727, Accuracy: 0.5714
Epoch [9/10], Loss: 0.0000, Accuracy: 0.0000
Epoch [10/10], Loss: 0.0360, Accuracy: 0.9286
